# RMS selectobjects profile: 

I think for the rms reactor, the main loop that we will want to worry about is the one in EdgeAnalysis.jl, "identify_objects", line  676: 

```
for i = 1:length(edgespeciesrateratios)
    if @inbounds  edgespeciesrateratios[i] > maxedgespeciesrateratios[i]
        @inbounds maxedgespeciesrateratios[i] = edgespeciesrateratios[i]
    end
end
```

this line iterates through all of the edge species. 

fyi rms reactors are in the rmgpy/rmg/reactors.py module. 

In [3]:
pathof(ReactionMechanismSimulator)

"/Users/blais.ch/Documents/_01_code/RMG_env_1/ReactionMechanismSimulator.jl/src/ReactionMechanismSimulator.jl"

In [1]:
using ReactionMechanismSimulator
using Profile
using PProf

In [2]:
minimal_core = "./rms_input/minimal.rms"
minimal_edge = "./rms_input/minimal_edge.rms"
outcore = readinput(minimal_core)
outedge = readinput(minimal_edge)
Profile.clear()

# minimal_core = "../02_profiling_examples/minimal/chemkin/chem_annotated.inp"
# minimal_core_dict = "../02_profiling_examples/minimal/chemkin/species_dictionary.txt"
# minimal_edge = "../02_profiling_examples/minimal/chemkin/chem_edge_annotated.inp"
# minimal_core_dict = "../02_profiling_examples/minimal/chemkin/species_edge_dictionary.txt"
# outcore = readinput(minimal_core;
#               spcdict=minimal_core_dict)
# outedge = readinput(minimal_edge;
#               spcdict=minimal_edge_dict)
corespcs = outcore["phase"]["Species"]
edgeishspcs = outedge["phase"]["Species"]
corerxns = outcore["phase"]["Reactions"]
edgeishrxns = outedge["phase"]["Reactions"]
coreedgespcs = copy(corespcs)

index = length(coreedgespcs)
for spc in edgeishspcs
    if nothing === findfirst(x->x.name==spc.name,coreedgespcs)
        newspc = Species(;name=spc.name,index=index+1,inchi=spc.inchi,smiles=spc.smiles,
            adjlist=spc.adjlist,thermo=spc.thermo,atomnums=spc.atomnums,diffusion=spc.diffusion,
            radius=spc.radius,radicalelectrons=spc.radicalelectrons,molecularweight=spc.molecularweight)
        global index += 1
        push!(coreedgespcs,newspc)
    end
end
coreedgespcsnames = getfield.(coreedgespcs,:name)

coreedgerxns = copy(corerxns)
index = length(coreedgerxns)
for rxn in edgeishrxns
    out = findfirst(x->getrxnstr(x)==getrxnstr(rxn),coreedgerxns)
    if nothing === out
        inds = findall(x->getrxnstr(x)==getrxnstr(rxn),edgeishrxns)
        for ind in inds
            rxnout = edgeishrxns[ind]
            reactants = [coreedgespcs[findfirst(x->spc.name==x,coreedgespcsnames)] for spc in rxnout.reactants]
            products = [coreedgespcs[findfirst(x->spc.name==x,coreedgespcsnames)] for spc in rxnout.products]
            reactantinds = [findfirst(x->spc.name==x,coreedgespcsnames) for spc in rxnout.reactants]
            productinds = [findfirst(x->spc.name==x,coreedgespcsnames) for spc in rxnout.products]
            newrxn = ElementaryReaction(;index=index+1,reactants=reactants,reactantinds=reactantinds,products=products,
                productinds=productinds,kinetics=rxnout.kinetics,electronchange=rxnout.electronchange,
                radicalchange=rxnout.radicalchange,reversible=rxnout.reversible,forwardable=rxnout.forwardable,pairs=rxnout.pairs)
            push!(coreedgerxns,newrxn)
        end
    end
end

coregas = IdealGas(corespcs,corerxns);
coreedgegas = IdealGas(coreedgespcs,coreedgerxns);

initialconds = Dict(["T"=>1350.0,"P"=>1.0e5,"ethane"=>1.0]);
spc = coregas.species[5] #ethane
termination = [TerminationConversion(spc,0.9),TerminationTime(1e6)];
coredomain,y0,corep = ConstantTPDomain(phase=coregas,initialconds=initialconds);
react = Reactor(coredomain,y0,(0.0,1e6);p=corep);
coreedgedomain,coreedgey0,coreedgep = ConstantTPDomain(phase=coreedgegas,initialconds=initialconds);
reactedge = Reactor(coreedgedomain,coreedgey0,(0.0,1e6);p=coreedgep);
@profile (terminated,resurrected,invalidobjects,unimolecularthreshold,bimolecularthreshold,trimolecularthreshold,maxedgespeciesrateratios) = selectobjects(react,reactedge,coreedgedomain,[],coredomain,
    [],corep,coreedgep,0.03,Inf,0.03,false,true,5,0.005,1.0,1.0,true,termination,1.0e8,Dict(),20,Inf)

# (terminated,resurrected,invalidobjects,unimolecularthreshold,bimolecularthreshold,trimolecularthreshold,maxedgespeciesrateratios) = selectobjects(react,reactedge,coreedgedomain,[],coredomain,
#     [],corep,coreedgep,0.03,Inf,0.03,false,true,5,0.005,1.0,1.0,true,termination,1.0e8,Dict(),20,Inf)

[17:05:45] WARNING: not removing hydrogen atom without neighbors
[17:05:46] WARNING: not removing hydrogen atom without neighbors
┌ Info: at time 0.0006143650423002915 sec, species [CH2]CCC at 0.03010496301839537 exceeded the minimum rate for simulation interruption of 0.03
└ @ ReactionMechanismSimulator /Users/blais.ch/Documents/_01_code/RMG_env_1/ReactionMechanismSimulator.jl/src/EdgeAnalysis.jl:761
┌ Info: At time 0.0006143650423002915 sec, species [CH2]CCC at rate ratio 0.03010496301839537 exceeded the minimum rate for moving to model core of 0.03
└ @ ReactionMechanismSimulator /Users/blais.ch/Documents/_01_code/RMG_env_1/ReactionMechanismSimulator.jl/src/EdgeAnalysis.jl:932
┌ Info: Terminating simulation due to interrupt
└ @ ReactionMechanismSimulator /Users/blais.ch/Documents/_01_code/RMG_env_1/ReactionMechanismSimulator.jl/src/EdgeAnalysis.jl:955


(false, false, Any[Species{NASA{EmptyThermoUncertainty}, StokesDiffusivity{Float64}, EmptyHenryLawConstant, EmptyLiquidVolumetricMassTransferCoefficient}("[CH2]CCC", 34, "", "[CH2]CCC", "", NASA{EmptyThermoUncertainty}
  polys: Array{NASApolynomial}((2,))
  unc: EmptyThermoUncertainty EmptyThermoUncertainty()
, Dict("C" => 4, "H" => 9), -1, StokesDiffusivity{Float64}
  r: Float64 2.954725557827581e-10
, 2.954725557827581e-10, 1, 0.05711600000000002, EmptyHenryLawConstant(), EmptyLiquidVolumetricMassTransferCoefficient(), "", false, false)], Bool[0, 0, 0, 0, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; … ;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], [9.527674716087324e-5, 4

In [3]:
pprof()

"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599
pprof: listen tcp 127.0.0.1:57599: bind: address already in use


In [5]:
length(corespcs), length(edgeishspcs),  length(corerxns),  length(edgeishrxns)

(26, 201, 71, 462)

"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599
